In [6]:
#titanic_kfp_pipeline.ipynb
#Copyright 2020 Google LLC. 
#This software is provided as-is, without warranty or representation for any use or purpose. 
#Your use of it is subject to your agreements with Google.
#Author: whjang@google.com

PROJECT_ID = 'kubeflow-test'
IMAGE_PREFIX = 'whjang-titanic'
PREPROC_DIR = 'preprocess'
TRAIN_DIR = 'train'

WORK_BUCKET = 'gs://kubeflow-test-kubeflowpipelines-default'
RAW_CSV_GCS_URI = WORK_BUCKET + '/rawdata/train.csv'
PREPROC_CSV_GCS_URI = WORK_BUCKET + '/preprocdata/processed_train.csv'
ACC_CSV_GCS_URI = WORK_BUCKET + '/latestacc/accuracy.csv'
MODEL_PKL_GCS_URI = WORK_BUCKET + '/model/model.pkl'
STAGE_GCS_FOLDER = WORK_BUCKET + '/stage'

AIPJOB_TRAINER_GCS_PATH = WORK_BUCKET + '/train/titanic_train.tar.gz'
AIPJOB_OUTPUT_GCS_PATH = WORK_BUCKET + '/train/output/'

import os
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["IMAGE_PREFIX"] = IMAGE_PREFIX
os.environ["PREPROC_DIR"] = PREPROC_DIR
os.environ["TRAIN_DIR"] = TRAIN_DIR
os.environ["WORK_BUCKET"] = WORK_BUCKET
os.environ["RAW_CSV_GCS_URI"] = RAW_CSV_GCS_URI
os.environ["PREPROC_CSV_GCS_URI"] = PREPROC_CSV_GCS_URI
os.environ["ACC_CSV_GCS_URI"] = ACC_CSV_GCS_URI
os.environ["MODEL_PKL_GCS_URI"] = MODEL_PKL_GCS_URI
os.environ["STAGE_GCS_FOLDER"] = STAGE_GCS_FOLDER
os.environ["AIPJOB_TRAINER_GCS_PATH"] = AIPJOB_TRAINER_GCS_PATH
os.environ["AIPJOB_OUTPUT_GCS_PATH"] = AIPJOB_OUTPUT_GCS_PATH

In [7]:
!docker build -t $IMAGE_PREFIX-$PREPROC_DIR $PREPROC_DIR/.
!docker tag $IMAGE_PREFIX-$PREPROC_DIR:latest docker.io/insoopark/$IMAGE_PREFIX-$PREPROC_DIR:latest
!docker push docker.io/insoopark/$IMAGE_PREFIX-$PREPROC_DIR:latest

Sending build context to Docker daemon  8.704kB
Step 1/7 : FROM frolvlad/alpine-python-machinelearning
 ---> 9e141b2fee53
Step 2/7 : RUN pip install --upgrade gcsfs argparse
 ---> Using cache
 ---> fcdbc53d4b53
Step 3/7 : ENV PYTHONUNBUFFERED 1
 ---> Using cache
 ---> ef9e9b665d43
Step 4/7 : RUN mkdir -p /titanic/src
 ---> Using cache
 ---> 409eecd5404e
Step 5/7 : COPY . /titanic/src
 ---> Using cache
 ---> d48eb57406bc
Step 6/7 : WORKDIR /titanic/src
 ---> Using cache
 ---> 5a8dcb43b05d
Step 7/7 : ENTRYPOINT ["python", "titanic_preprocess.py"]
 ---> Using cache
 ---> 3c5a2d6c7b8b
Successfully built 3c5a2d6c7b8b
Successfully tagged whjang-titanic-preprocess:latest
The push refers to repository [docker.io/insoopark/whjang-titanic-preprocess]

34e95718: Preparing 
2d7b02ac: Preparing 
7234986d: Preparing 
2d9188d2: Preparing 
01e901e5: Preparing 
9ef47504: Preparing 
9ef47504: Layer already exists latest: digest: sha256:32b2645a1a8dac0046467b8c0c9b39385560c68d6c9c259013ec64119f83590e siz

In [8]:
!docker run docker.io/insoopark/$IMAGE_PREFIX-$PREPROC_DIR:latest --raw_csv_gcs_uri $RAW_CSV_GCS_URI --preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI

Preprocessing Titanic Data
   Pclass  SibSp  Sex_female  Sex_male  Survived
0       3      1           0         1         0
1       1      1           1         0         1
2       3      0           1         0         1
3       1      1           1         0         1
4       3      0           0         1         0


In [16]:
!docker build -t $IMAGE_PREFIX-$TRAIN_DIR $TRAIN_DIR/.
!docker tag $IMAGE_PREFIX-$TRAIN_DIR:latest docker.io/insoopark/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker push docker.io/insoopark/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker run docker.io/insoopark/$IMAGE_PREFIX-$TRAIN_DIR:latest \
--preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI \
--model_pkl_gcs_uri $MODEL_PKL_GCS_URI \
--acc_csv_gcs_uri $ACC_CSV_GCS_URI \
--min_acc_progress 0.000001

Sending build context to Docker daemon  18.94kB
Step 1/7 : FROM frolvlad/alpine-python-machinelearning
 ---> 9e141b2fee53
Step 2/7 : RUN pip3 install google-cloud-storage==1.0.0 gcsfs sklearn argparse simplejson
 ---> Running in cd63a60eddb1
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=f529bd56002ec80b9f070a21b992c0184b66f5f9ab27ed4adbb7e87d00df367f
  Stored in directory: /root/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
  Created wheel for simplejson: filename=simplejson-3.17.2-py3-none-any.whl size=55459 sha256=5fdc79c12e0b7d801384261c0b6a7057bef436eac516cbf73bb01b391a850493
  Stored in directory: /root/.cache/pip/wheels/17/72/7d/df0984c925921e22322ea462a6f861e9d0617881192deb9b8d
Successfully built sklearn simplejson
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Removing intermediate container cd63a60eddb1
 ---> 739687dc0b18
Step 3/7 : ENV PYTHONUNBUFFERED 1
 

In [17]:
!rm -fr titanic_train.tar.gz
!tar zcvf titanic_train.tar.gz *
!gsutil cp titanic_train.tar.gz $AIPJOB_TRAINER_GCS_PATH

handson.ipynb
katib_mnist.py
preprocess/
preprocess/.ipynb_checkpoints/
preprocess/.ipynb_checkpoints/Dockerfile-checkpoint
preprocess/.ipynb_checkpoints/titanic_preprocess-checkpoint.py
preprocess/titanic_preprocess.py
preprocess/Dockerfile
runjob.sh
setup.py
titanic_kfp_pipeline.ipynb
train/
train/.ipynb_checkpoints/
train/.ipynb_checkpoints/titanic_train-checkpoint.py
train/.ipynb_checkpoints/Dockerfile-checkpoint
train/.ipynb_checkpoints/__init__-checkpoint.py
train/titanic_train.py
train/__init__.py
train/Dockerfile
Copying file://titanic_train.tar.gz [Content-Type=application/x-tar]...
/ [1 files][ 23.5 KiB/ 23.5 KiB]                                                
Operation completed over 1 objects/23.5 KiB.                                     


In [ ]:
!pip3 install -U kfp
import kfp
import kfp.components as comp
from kfp import dsl
from kfp import compiler
from kfp.components import func_to_container_op
import time
import datetime

PIPELINE_HOST = "45b30b46974186c0-dot-us-central2.pipelines.googleusercontent.com"
WORK_BUCKET = "gs://kubeflow-test-kubeflowpipelines-default"
EXPERIMENT_NAME = "Titanic Draft Experiment"

# Function for determine deployment
@func_to_container_op
def check_and_deploy_op(ACC_CSV_GCS_URI) -> str:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "pandas"])
    subprocess.run([sys.executable, "-m", "pip", "install", "gcsfs"])
    import pandas as pd

    print("ACC_CSV_GCS_URI: ", ACC_CSV_GCS_URI)
    
    acc_df = pd.read_csv(ACC_CSV_GCS_URI)
    return acc_df["deploy"].item()

@func_to_container_op
def finish_deploy_op(ACC_CSV_GCS_URI):
 import sys, subprocess
 subprocess.run([sys.executable, "-m", "pip", "install", "pandas"])
 subprocess.run([sys.executable, "-m", "pip", "install", "gcsfs"])
 import pandas as pd
 acc_df = pd.read_csv(ACC_CSV_GCS_URI)
 acc_df["deploy"] = "done"
 acc_df.to_csv(ACC_CSV_GCS_URI)
 print("Successfully new model was deployed")
    
@dsl.pipeline(
 name="titanic-kubeflow-pipeline-demo",
 description = "Titanic Kubeflow Pipelines demo embrassing AI Platform in Google Cloud"
)

def titanic_pipeline(
 PROJECT_ID,
 WORK_BUCKET,
 RAW_CSV_GCS_URI,
 PREPROC_CSV_GCS_URI,
 ACC_CSV_GCS_URI,
 MODEL_PKL_GCS_URI,
 MIN_ACC_PROGRESS,
 STAGE_GCS_FOLDER,
 TRAIN_ON_CLOUD,
 AIPJOB_TRAINER_GCS_PATH,
 AIPJOB_OUTPUT_GCS_PATH
):
    IMAGE_PREFIX = "whjang-titanic"
    PREPROC_DIR = "preprocess"
    TRAIN_DIR = "train"
    MODEL_DIR = "model"
 
    preprocess = dsl.ContainerOp(
        name = "Preprocess raw data and generate new one",
        #image = "gcr.io/" + str(PROJECT_ID) + "/" + IMAGE_PREFIX + "-" + PREPROC_DIR + ":latest",
        image = "docker.io/insoopark/" + IMAGE_PREFIX + "-" + PREPROC_DIR + ":latest",
        arguments = [
            "--raw_csv_gcs_uri", RAW_CSV_GCS_URI,
            "--preproc_csv_gcs_uri", PREPROC_CSV_GCS_URI
        ]
    )
    
    train_args = [
        "--preproc_csv_gcs_uri", str(PREPROC_CSV_GCS_URI),
        "--model_pkl_gcs_uri", str(MODEL_PKL_GCS_URI),
        "--acc_csv_gcs_uri", str(ACC_CSV_GCS_URI),
        "--min_acc_progress", str(MIN_ACC_PROGRESS)
    ]
 
    with dsl.Condition(TRAIN_ON_CLOUD == False) as check_condition1:
        train = dsl.ContainerOp(
            name = "Train",
            image = "docker.io/insoopark/" + IMAGE_PREFIX + "-" + TRAIN_DIR + ":latest",
            arguments = train_args,
                file_outputs={
                    "mlpipeline-metrics" : "/mlpipeline-metrics.json"
                }
        )
 
    # added by ispark
    import json
    with dsl.Condition(TRAIN_ON_CLOUD == True) as check_condition2:
        aip_job_train_op = comp.load_component_from_url("https://raw.githubusercontent.com/kubeflow/pipelines/1.0.0/components/gcp/ml_engine/train/component.yaml")
        help(aip_job_train_op)
        aip_train = aip_job_train_op(
            project_id=PROJECT_ID, 
            python_module="train.titanic_train", 
            package_uris=json.dumps([str(AIPJOB_TRAINER_GCS_PATH)]), 
            region="us-west1", 
            args=json.dumps(train_args),
            job_dir=AIPJOB_OUTPUT_GCS_PATH, 
            python_version="3.7",
            runtime_version="1.15", #cf. 2.1 
            master_image_uri="", 
            worker_image_uri="", 
            training_input="", 
            job_id_prefix="", 
            job_id="",
            wait_interval=5
        )
 
    check_deploy = check_and_deploy_op(ACC_CSV_GCS_URI)
    with dsl.Condition(check_deploy.output == "pending"):
        aip_model_deploy_op = comp.load_component_from_url("https://raw.githubusercontent.com/kubeflow/pipelines/1.0.0/components/gcp/ml_engine/deploy/component.yaml")
        help(aip_model_deploy_op)
        aip_model_deploy = aip_model_deploy_op(
            model_uri=str(WORK_BUCKET) + "/" + MODEL_DIR, 
            project_id=PROJECT_ID, 
            model_id="", 
            version_id="", 
            runtime_version="1.15", #cf. 2.1 
            python_version="3.7",
            version="", 
            replace_existing_version="False", 
            set_default="True", 
            wait_interval=5
        )
        lastStep = finish_deploy_op(ACC_CSV_GCS_URI)
 
    check_condition1.after(preprocess)
    check_condition2.after(preprocess)
    check_deploy.after(aip_train)
    lastStep.after(aip_model_deploy)
 
    train.execution_options.caching_strategy.max_cache_staleness = "P0D"
    aip_train.execution_options.caching_strategy.max_cache_staleness = "P0D"
    check_deploy.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    aip_model_deploy.execution_options.caching_strategy.max_cache_staleness = "P0D"
    lastStep.execution_options.caching_strategy.max_cache_staleness = "P0D"
 
args = {
    "PROJECT_ID" : "aiplatformdemo",
    "WORK_BUCKET" : WORK_BUCKET,
    "RAW_CSV_GCS_URI" : WORK_BUCKET + "/rawdata/train.csv",
    "PREPROC_CSV_GCS_URI" : WORK_BUCKET + "/preprocdata/processed_train.csv",
    "ACC_CSV_GCS_URI" : WORK_BUCKET + "/latestacc/accuracy.csv",
    "MODEL_PKL_GCS_URI" : WORK_BUCKET + "/model/model.pkl",
    "MIN_ACC_PROGRESS" : 0.000001,
    "STAGE_GCS_FOLDER" : WORK_BUCKET + "/stage",
    "TRAIN_ON_CLOUD" : False,
    "AIPJOB_TRAINER_GCS_PATH" : WORK_BUCKET + "/train/titanic_train.tar.gz",
    "AIPJOB_OUTPUT_GCS_PATH" : WORK_BUCKET + "/train/output/"
}

client = kfp.Client(host=PIPELINE_HOST)
#pipeline_name = “titanic_pipelines.zip”
#compiler.Compiler().compile(titanic_pipeline, pipeline_name)
#try:
# pipeline = client.upload_pipeline(pipeline_package_path=pipeline_name, pipeline_name=pipeline_name)
# print(“uploaded:” + pipeline.id)
#except:
# print(“already exist”)
client.create_run_from_pipeline_func(
    titanic_pipeline,
    arguments=args,
    experiment_name=EXPERIMENT_NAME
)